In [312]:
import numpy as np   #importing libraries
import simpy 
import queue
import random
import random as rd
import matplotlib 
from matplotlib import pyplot as plt

In [313]:
global SZ,CP,RM
SZ=[]
CP=[]
RM=[]
N=1000
k=10
Lambda = 1
mu = 1.25
#t = 0   #system clock
t1=0
t2=0
t_lambda = 0 #poisson random variable used to generate next service time
t_mu = 0  #poisson random variable used to generate next departure time
t_a=0
wait=0
R=1000
transmission_time=0
l="0"
P_transmission=500
P_idle=100
alpha = 0.1
gamma = 0.95
episodes = 10
SHOW_EVERY = 100
STATS_EVERY=100
observation_space_discretesize = [11, 11, 2]
q_table = np.random.uniform(low=-1, high=0, size=(observation_space_discretesize))


In [314]:
class Packet(object):
    def __init__(self, time, id, mobileID, seeed):
        
        self.time=time                                                
        self.id=id
        self.mobileID=mobileID   
        self.size=random.randint(200,400) #seed 1 op 2, seed 2 op 
        SZ.append(round(self.size,2))
        #print(seeed)
        #random.seed(seeed)
        self.cpu=random.randint(20,90) #seed 1 op 2, seed 2 op 
        CP.append(round(self.cpu,2))
        #random.seed(seeed)
        self.ram=random.randint(1, 32) #seed 1 op 3, seed 2 op 1, seed 3 op 4
        RM.append(round(self.ram,2))

class mobile(object): #mobile class with an ID              
    def __init__(self, ID):
        self.ID=ID
        self.pot=int(1)
        
    #generates packets and sends via out_pipe
    def packet_generator(self, numPackets, env, out_pipe, task_id,send): 
        
        yield env.timeout(send)
        #print("sending task {} of mobile {} at time {}".format(task_id, self.ID, env.now))
        p=Packet(env.now, task_id, self.ID, self.pot)
        self.pot+=1
        if(self.pot==4):
                self.pot=1
        yield out_pipe.put(p)
        item = yield out_pipe.get()



In [315]:
 def reward_generator(T): 
    Tsla=2.5 
    Nsla=3
    Gsla=3  
    Cost_sla=50
    Cg=0
    Ct=0
    Cn=0
    Cost_n=0
    
    #if (G<Gsla):
        #Cg=-5*(Gsla/G)
    if (T>Tsla):
        Ct = -5 * (T/Tsla)
    #if (N>Nsla):
        #Cn=-5*(N/Nsla)
    #if (cost>Cost_sla):
        #Cost_n=-5*(cost/Cost_sla)
    
    Reward = (Tsla/T) + Ct #Reward Function
    return Reward

In [316]:
class Fog(object):      
    def __init__(self, ID):
        self.ID=ID
        self.Number_queue=1
        self.lost=0
        self.response_time=0
        self.server_count=0
        self.wait=0
        self.taskinqueue=0
        self.a=0
        self.time=0
        self.neighbour1=None
        
        self.service_time=[]
        self.response_time_list=[]
        self.Serverqueue=[]
        self.total_serverqueue=[]
        self.waitlist=[]
        self.energy_offloading=[]
        self.reward_list=[]
        self.state=[]
        
    def Neighbour(self,neighbour):
        self.neighbour1=neighbour
        
    def FOG(self, env, server, in_pipe,mobile,pipe3,pipe4):
         
         msg = yield in_pipe.get()
         transmission_time = (msg.size/R)
         #print("transmits task {} to need time {}".format(msg.id,transmission_time,self.ID))
         yield env.timeout(transmission_time)
         self.a=env.now
         #print("Arrived task {} at time {} at FOG {}".format(msg.id,self.a,self.ID))
         #print("server",self.server_count)
         #print("present q",self.taskinqueue)
         discrete_state=()
         discrete_state=(self.taskinqueue,self.neighbour1.taskinqueue,)
         yield pipe4.put(discrete_state)
         if self.server_count==3:
            if len(self.Serverqueue)<k:
               self.Serverqueue.append(msg)
               self.taskinqueue=len(self.Serverqueue)
               #print("queue te ase ",len(self.Serverqueue))
               #print("task ase queue te ase ",taskinqueue)
               self.Number_queue = self.Number_queue + 1
               #self.server_count=0
            else:
                self.lost=self.lost+1 
                return  
         #s=q+1  
         #q=s
         u=rd.random()
         t_mu=-np.log(u)/mu
         self.service_time.append(t_mu)
         #print("service time",t_mu)
         E = (P_transmission * transmission_time) + (P_idle * t_mu)
         self.energy_offloading.append(E)
         
         #print("present a queue te ase ",len(self.Serverqueue))
         #print("Fog:   Fog",self.ID,"mobile",mobile,", akhon queue te ase ",self.taskinqueue,", time",env.now)

         with server.request() as req:
             yield req
             s=env.now
             self.wait=s-self.a
             if len(self.Serverqueue)>0:
                 self.Serverqueue.pop(0)
             #self.total_serverqueue.append(len(self.Serverqueue))
             #print("queue",len(self.Serverqueue))
             #print("wait",self.wait)
             #print("Starting service task {} at time {} at FOG {}".format(msg.id,s,self.ID))
             #print("Waiting time of task {} at time {} at FOG {}".format(msg.id,self.wait,self.ID))
             self.waitlist.append(self.wait)
             self.response_time=self.wait+t_mu
             self.response_time_list.append(self.response_time)
             
             #if self.wait>0:
                #print("queue",len(self.Serverqueue))
              #  if len(self.Serverqueue)<10:
               #     self.Serverqueue.append(msg)
                #    self.Number_queue = self.Number_queue + 1
                 #   self.total_serverqueue.append(len(self.Serverqueue))
                  #  yield env.timeout(t_mu)
                   # print("Finish service task {} at time {} at FOG {}".format(msg.id,env.now,self.ID))
                    #if len(self.Serverqueue)>0:
                     #   self.Serverqueue.pop(0)
                #else:
                 #   self.lost=self.lost+1 
                  #  return 
             self.server_count=self.server_count+1
             yield env.timeout(t_mu)
             #print("Finish service task {} at time {} at FOG {}".format(msg.id,env.now,self.ID))
             self.server_count=self.server_count-1   

                
             r=reward_generator(self.response_time)
             self.reward_list.append(r)
                
             discrete_state2=()
             #print("next q",self.taskinqueue)
             #state.append([self.ID,mobile,self.taskinqueue,env.now,self.neighbour1.ID,self.neighbour1.taskinqueue])
             discrete_state2=(self.taskinqueue,self.neighbour1.taskinqueue,)
             #yield pipe3.put(state)
             yield pipe3.put(discrete_state2)
             #print("Fog",self.ID,"pore queue te ase ",self.taskinqueue,"time hoitese",env.now)
    
    def nextstate(self,env,mobile,pipe3,pipe4,statepipe,action):
        discrete_state=yield pipe4.get()
        next_state = yield pipe3.get()
        #print("FOG, mobile, queue te ase, time, neighbour FOG, neighbour queue",state)
        #print("present state",discrete_state)
        #print("next state",next_state)
        max_future_q = np.max(q_table[next_state])
        current_q = q_table[discrete_state + (action,)]
        new_q = current_q + alpha * (3 + gamma * max_future_q - current_q )
        q_table[discrete_state + (action,)] = new_q
        yield statepipe.put(discrete_state)
        return discrete_state
        #discrete_state = next_state

        #next_state=self.taskinqueue
        #print("nxtstate: Fog",self.ID,"mobile",mobile,", req er age queue te ase",next_state,", time",env.now)
        

In [317]:
class Decision(object):
    def __init__(self, ID):
        self.ID=ID
        self.fog1_count=0
        self.fog2_count=0
        self.t1=0
        self.t2=0
        self.current=()
    def decision(self,env1,server1,server2,pipe1,pipe2,pipe3,pipe4,statepipe,ms1,ms2,fog1,fog2,i):
        yield env1.timeout(1)
        fog1.Neighbour(fog2)
        fog2.Neighbour(fog1)
        
        if(i==0):
            self.fog1_count=self.fog1_count+1           
            u=rd.random()
            t_lambda=-np.log(u)/Lambda
            t_a=self.t1+t_lambda
            self.t1=t_a
            env1.process(ms1.packet_generator(1, env1, pipe1, 0,t_a))
            env1.process(fog1.FOG(env1, server1, pipe1,ms1.ID,pipe3,pipe4))
            action=random.randint(0,1)
            self.current=yield env1.process(fog1.nextstate(env1,ms1.ID,pipe3,pipe4,statepipe,action))
            print("current state",self.current)
            #current=yield statepipe.get()
            #print("current state",current)
        if(i>0):
            action=random.randint(0,1)

    #             self.fog1_count=self.fog1_count+1           
    #             u=rd.random()
    #             t_lambda=-np.log(u)/Lambda
    #             t_a=self.t1+t_lambda
    #             self.t1=t_a
    #             env1.process(ms1.packet_generator(1, env1, pipe1, i,t_a))
    #             env1.process(fog1.FOG(env1, server1, pipe1,ms1.ID,pipe3,pipe4))

            env1.process(fog1.nextstate(env1,ms1.ID,pipe3,pipe4,statepipe,action))
            #current=yield statepipe.get()
                #print("current state",current)

            if(action==0):
                    self.fog1_count=self.fog1_count+1
                    u=rd.random()
                    t_lambda=-np.log(u)/Lambda
                    t_a=self.t1+t_lambda
                    self.t1=t_a
                    #print("ms1,fog1",self.t1)
                    env1.process(ms1.packet_generator(1, env1, pipe1, i,t_a))
                    env1.process(fog1.FOG(env1, server1, pipe1,ms1.ID,pipe3,pipe4))
                    #env1.process(fog1.nextstate(env1,ms1.ID,pipe3,pipe4,action))

                    self.fog1_count=self.fog1_count+1
                    u=rd.random()
                    t_lambda=-np.log(u)/Lambda
                    t_a=self.t2+t_lambda
                    self.t2=t_a
                    #print("ms2,fog1",self.t1)
                    env1.process(ms2.packet_generator(1, env1, pipe2, i,t_a))
                    env1.process(fog1.FOG(env1, server1, pipe2,ms2.ID,pipe3,pipe4))
                    #env1.process(fog1.nextstate(env1,ms2.ID,pipe3,pipe4,action))
            if(action==1):
                     self.fog2_count=self.fog2_count+1
                     u=rd.random()
                     t_lambda=-np.log(u)/Lambda
                     t_a=self.t1+t_lambda
                     self.t1=t_a
                     #print("ms1,fog2",self.t1)
                     env1.process(ms1.packet_generator(1, env1, pipe1, i,t_a))
                     env1.process(fog2.FOG(env1, server2, pipe1,ms1.ID,pipe3,pipe4))
                     #env1.process(fog2.nextstate(env1,ms1.ID,pipe3,pipe4,action))

                     self.fog2_count=self.fog2_count+1
                     u=rd.random()
                     t_lambda=-np.log(u)/Lambda
                     t_a=self.t2+t_lambda
                     self.t2=t_a
                     #print("ms1,fog2",self.t1)
                     env1.process(ms2.packet_generator(1, env1, pipe2, i,t_a))
                     env1.process(fog2.FOG(env1, server2, pipe2,ms2.ID,pipe3,pipe4))
                     #env1.process(fog2.nextstate(env1,ms2.ID,pipe3,pipe4,action))

            env1.process(fog1.nextstate(env1,ms1.ID,pipe3,pipe4,statepipe,action))
            self.current=yield statepipe.get()
            print("current state",self.current)
            #env1.run()


            #i=0
            #while (i<N):
             #   print("{}     {}     {}".format(i+1,fog1.response_time_list[i],fog1.reward_list[i]))
             #   i+=1

            #print(fog1.lost,fog2.lost)
            #print(fog1_count,fog2_count)
            #print(len(fog1.total_serverqueue))
            #Na=N-fog.lost
            #print("No    serverqueue       wait_time")
            #i=0
            #c=0
            #while (i<fog2_count):
               # if(fog2.waitlist[i]>0):
              #      c+=1
             #   i+=1
            #print("total queue te ase",fog2.Number_queue)
            #print("total wait list e ase",c)
            #print("FOG-1 MAXIMUM REAWARD:", max(fog1.reward_list))
            #print("FOG-1 total number of task in queue",fog1.Number_queue)


In [318]:
env1 = simpy.Environment()    
server1 = simpy.Resource(env1, capacity=3)
server2 = simpy.Resource(env1, capacity=3)

pipe1 = simpy.Store(env1)
pipe2 = simpy.Store(env1)
pipe3 = simpy.Store(env1)
pipe4 = simpy.Store(env1)
statepipe = simpy.Store(env1)

ms1=mobile(1)
ms2=mobile(2)
ms3=mobile(3)

fog1 = Fog(1)
fog2 = Fog(2)

obj=Decision(1)
for i in range(N+1):
    env1.process(obj.decision(env1,server1,server2,pipe1,pipe2,pipe3,pipe4,statepipe,ms1,ms2,fog1,fog2,i))
env1.run()

fog1_count=obj.fog1_count
fog2_count=obj.fog2_count
print("count",fog1_count,fog2_count)
avgwaittime=sum(fog1.waitlist)/fog1_count
print("FOG-1 Average queueing time:", avgwaittime)
avgservicetime=sum(fog1.service_time)/fog1_count
print("FOG-1 Average service time:", avgservicetime)
avgsystemtime=sum(fog1.response_time_list)/fog1_count
print("FOG-1 Average response time:", avgsystemtime)
avgenergy=sum(fog1.energy_offloading)/fog1_count
print("FOG-1 Average Energy consumption:", avgenergy)
#print("No    response_time       reward")
#i=0
#while (i<N):
#   print("{}     {}     {}".format(i+1,fog2.response_time_list[i],fog2.reward_list[i]))
#  i+=1
#print("FOG-2 MAXIMUM REAWARD:", max(fog2.reward_list))
#print("FOG-2 total number of task in queue",fog2.Number_queue)
avgwaittime=sum(fog2.waitlist)/fog2_count
print("FOG-2 Average queueing time:", avgwaittime)
avgservicetime=sum(fog2.service_time)/fog2_count
print("FOG-2 Average service time:", avgservicetime)
avgsystemtime=sum(fog2.response_time_list)/fog2_count
print("FOG-2 Average response time:", avgsystemtime)
avgenergy=sum(fog2.energy_offloading)/fog2_count
print("FOG-2 Average Energy consumption:", avgenergy)


current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state (0, 0)
current state